In [3]:
# import stuff
import sys,os
import numpy as np


sys.path.append( os.environ["PYTHONSCRIPT_DIR"] )

import matplotlib as mpl
import PlotSettings as pS
pS.loadPlotSettings(mpl)

dLineSettings = pS.defaultLineSettings 
dMarkerSize = pS.defaultMarkerSize

from Tools.Parsers.Utilities import parseNDArray

sys.path.append( os.environ["JOBGENERATOR_DIR"])
from HPCJobConfigurator.jobGenerators.commonFunctions import CommonFunctions as cf 


import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors

from skimage.io import imread
from skimage.filters import *
from skimage.morphology import *
from skimage import data
from skimage import img_as_float
from skimage.morphology import disk


#GUi shit
%matplotlib tk

# Import General Packages from me
from Tools.Parsers import *
from Tools.BoundingBox import *

%load_ext autoreload
%autoreload 2



Setting matplotlib settings from file: /home/zfmgpu/Desktop/Repository/SimulationFramework/SourceCode/Projects/SimulationFramework/Simulations/PythonScripts/PlotSettings/matplotlibrc
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
pS.distinctColors.keys()

dict_keys(['blackredgreen', 'blackredgreenlight', 'colors3light', 'colors3'])

In [4]:
# plot dynState
from mpldatacursor import datacursor

def plotData(dynState):
    plt.close('all')
    
    
    fig, (ax0,ax1,ax2) = plt.subplots(nrows = 3);
    n = len(dynState.bodies)
    jet = plt.get_cmap('jet') 
    cNorm  = colors.Normalize(vmin=0, vmax=n)
    scalarMap = cm.ScalarMappable(norm=cNorm, cmap=jet)
    
    
    # Position
    
    for i,body in dynState.bodies.items():
        linex, = ax0.plot(dynState.t, body[0][:,0],'.-', color= scalarMap.to_rgba(i), label='x , b: ' + str(i));
        liney, = ax0.plot(dynState.t, body[0][:,1],'.-', color= scalarMap.to_rgba(i), label='y , b: ' + str(i));
        linez, = ax0.plot(dynState.t, body[0][:,2],'.-', color= scalarMap.to_rgba(i), label='z , b: ' + str(i));
        datacursor([linex,liney,linez],draggable=True)
        ax1.plot(dynState.t, body[0][:,3:7],'.-', color= scalarMap.to_rgba(i))
        ax2.plot(dynState.t, body[2]['rank'][:,0],'.-', color= scalarMap.to_rgba(i));
        
    
    
    ax0.grid(True)
    ax0.set_title('Position');
    ax0.set_ylabel('\mathbf{r}_S(t)');
    ax0.set_ymargin(0.2)
    ax0.autoscale(True,'both',False)
    
    ax1.grid(True)
    ax1.set_title('Quaternion');
    ax1.set_ylabel('\mathbf{a}_{KI}(t)');
    ax1.set_ymargin(0.2)
    ax1.autoscale(True,'both',False)
    
    ax2.grid(True)
    ax2.set_title('Rank');
    ax2.set_ylabel('rank number');
    ax2.set_ymargin(0.2)
    ax2.autoscale(True,'both',False)
    
    
    # Velocity
    fig2, (ax20,ax21) = plt.subplots(nrows = 2);
    
    
    
    for i,body in dynState.bodies.items():               
        ax20.plot(dynState.t, body[1][:,0:3], '.-' , color= scalarMap.to_rgba(i));
        ax21.plot(dynState.t, body[1][:,3:6], '.-' , color= scalarMap.to_rgba(i));
    
    ax20.grid(True)
    ax20.set_title('Velocity');
    ax20.set_ylabel('\mathbf{v}_S(t)');
    ax20.set_xmargin(0.0)
    ax20.set_ymargin(0.1)
    
    ax21.grid(True)
    ax21.set_title('Angular Velocity');
    ax21.set_ylabel('\mathbf{\omega}_{KI}(t)');
    ax21.set_xmargin(0.0)
    ax21.set_ymargin(0.1)
    return ( (fig,(ax0,ax1,ax2)) , (fig2, (ax20,ax21) )  );

def plotData3D(dynState):
    pass
        

Plot SimData.dat for MPI
---

In [34]:
# plot SimData MPI

import re, os, copy

class PlotSimDataMPI(SimDataReader.SimDataConsolidate):
    
    def __init__(self):
        self.axarr = [];
        self.figarr = [];
    
    def plot(self,figOpts=None,plotOpts=None):
        
        self.figOpts = dict(
                            oneWindow=False,
                            figSize=(pS.cm2inch(8),pS.cm2inch(6)),
                            showTitle = True
                           )
        
        self.plotOpts = dict(
            legendProps = dict(prop=dict(size=8),borderpad=0.2,labelspacing=0.1)
        )
        
        import collections
        def update(d, u):
            for k, v in u.items():
                if isinstance(v, collections.Mapping):
                    r = update(d.get(k, {}), v)
                    d[k] = r
                else:
                    d[k] = u[k]
            return d
        
        if figOpts is not None:
            self.figOpts = update(self.figOpts , figOpts)
        if plotOpts is not None:
            self.plotOpts = update(self.plotOpts , plotOpts)
            
        # Plot stuff
        self.generatePlotsAndAxes()
        self.plotSimDataMPI(self.procIndices,self.simDataList)
        
#         if(self.simDataGuiPath):
#             self.plotComparisionGUI(simDataGuiPath)

        for f in self.figarr:
             f.canvas.draw()
                
        
    
    def save(self,file, **kwargs):
        for i,fig in enumerate(self.figarr):
            fig.savefig(file%i,**kwargs)
    
    def generatePlotsAndAxes(self):
        
        # only make figures if not already added
        if len(self.figarr) == 0:
            
            if not self.figOpts["oneWindow"]:

                for i in range(0,9):
                    f = plt.figure(figsize=self.figOpts["figSize"])
                    self.figarr.append(f);
                    ax = f.add_subplot(111);
                    self.axarr.append(ax);

            else:
                fig, axarr = plt.subplots(3,3, sharex=True)
                self.figarr.append(fig);
                self.axarr.extend(list(axarr.flatten()));

            self.axdict = dict();
            self.grids = []
            
            timelabel = r'simulation time $t_k$ [s]'
            
            self.axdict['TotalStateEnergy']= self.axarr[0];
            self.axdict['TotalStateEnergy'].ticklabel_format(style='sci', axis='y', scilimits=(0,0))
            #axdict['TotalStateEnergy'].set_xlabel('time [s]');
            self.axdict['TotalStateEnergy'].set_ylabel('total energy $T+V$ [J]');
            self.axdict['TotalStateEnergy'].set_xlabel(timelabel);

            self.axdict['nGlobalIterations'] = self.axarr[1];
            #axdict['nGlobalIterations'].set_xlabel('time [s]');
            self.axdict['nGlobalIterations'].set_ylabel('proximal iterations');
            self.axdict['nGlobalIterations'].set_xlabel(timelabel);

            self.axdict['Converged'] = self.axarr[2];
            #axdict['Converged'].set_xlabel('time [s]');
            self.axdict['Converged'].set_ylabel('iteration converged [0,1]');
            self.axdict['Converged'].set_xlabel(timelabel);

            self.axdict['nContacts'] = self.axarr[3];
            self.axdict['nContacts'].ticklabel_format(style='sci', axis='y', scilimits=(0,0))
            #axdict['nContacts'].set_xlabel('time [s]');
            self.axdict['nContacts'].set_ylabel('number of contacts');
            self.axdict['nContacts'].set_xlabel(timelabel);
            self.axdict['nContacts'].ticklabel_format(style='sci', axis='y', scilimits=(0,0))

            self.axdict['nContactsAll'] = self.axarr[4];
            self.axdict['nContactsAll'].ticklabel_format(style='sci', axis='y', scilimits=(0,0))
            #axdict['nContacts'].set_xlabel('time [s]');
            self.axdict['nContactsAll'].set_ylabel('number of contacts');
            self.axdict['nContactsAll'].set_xlabel(timelabel);

            self.axdict['nContactsAllRL'] = self.axarr[5];
            #axdict['nContacts'].set_xlabel('time [s]');
            self.axdict['nContactsAllRL'].set_ylabel('number of contacts');
            self.axdict['nContactsAllRL'].set_xlabel(timelabel);

            self.axdict['TimeStepTime'] = self.axarr[6];
            #axdict['TimeStepTime'].set_xlabel('time [s]');
            self.axdict['TimeStepTime'].set_ylabel('cpu time: time-step [s]');
            self.axdict['TimeStepTime'].set_xlabel(timelabel);

            self.axdict['MaxOverlap'] = self.axarr[7];
            self.axdict['MaxOverlap'].ticklabel_format(style='sci', axis='y', scilimits=(0,0))
            self.axdict['MaxOverlap'].ticklabel_format(style='sci', axis='y', scilimits=(0,0))
            #axdict['MaxOverlap'].set_xlabel('time [s]');
            self.axdict['MaxOverlap'].set_ylabel('maximal overlap [m]');
            self.axdict['MaxOverlap'].set_xlabel(timelabel);

            self.axdict['nSplitBodyNodes'] = self.axarr[8];
            self.axdict['nSplitBodyNodes'].set_xlabel(timelabel);
            self.axdict['nSplitBodyNodes'].set_ylabel('number of split-nodes');
            

            # Total plots:

            f = plt.figure(figsize=self.figOpts["figSize"]);
            ax = f.add_subplot(111);
            self.axdict['maxTimings'] = ax;
            ax.set_xlabel(timelabel);
            ax.set_ylabel('cpu time [s]');
            #ax.set_yscale("log")
            self.figarr.append(f);
            self.axarr.append(ax);


            # All timings
            f = plt.figure(figsize=self.figOpts["figSize"]);
            ax = f.add_subplot(111);
            self.axdict['allTimings'] = ax;
            ax.set_xlabel(timelabel);
            ax.set_ylabel('cpu time [s]');
            ax.set_yscale("log")
            self.figarr.append(f);
            self.axarr.append(ax);

            # Coll timings
            f = plt.figure(figsize=self.figOpts["figSize"]);
            ax = f.add_subplot(111);
            self.axdict['collisionTime'] = ax;
            ax.set_xlabel(timelabel);
            ax.set_yscale("log")
            ax.set_ylabel('cpu time: coll. solver [s]');
            self.figarr.append(f);
            self.axarr.append(ax);

            # Inc timings
            f = plt.figure(figsize=self.figOpts["figSize"]);
            ax = f.add_subplot(111);
            self.axdict['inclusionTime'] = ax;
            ax.set_xlabel(timelabel);
            ax.set_yscale("log")
            ax.set_ylabel('cpu time: incl. solver [s]');
            self.figarr.append(f);
            self.axarr.append(ax);


            # BodyComm timings
            f = plt.figure(figsize=self.figOpts["figSize"]);
            ax = f.add_subplot(111);
            self.axdict['bodyCommTime'] = ax;
            ax.set_xlabel(timelabel);
            ax.set_yscale("log")
            ax.set_ylabel('cpu time: body comm. [s]');
            self.figarr.append(f);
            self.axarr.append(ax);

            for ax in self.axdict.values():
                ax.set_axisbelow(True)
              
        

    def plotSimDataMPI(self,procIndices,simDataList):
        
        if procIndices is not None and procIndices: 
            nProcesses = len(procIndices);
            self.scalarMap = self.generateColorMap(nProcesses);

            for p in range(nProcesses):
                self.plotSimDataProcess(simDataList["All"],
                                        p,
                                        self.generateOptions("All", self.scalarMap, p)
                                       )         
        
        self.plotSimDataTotal(simDataList["Total"],
                              self.generateOptions("Total") );
        self.plotSimDataAverage(simDataList["Average"],
                                self.generateOptions("Average"));
     
        
        
        # add legends
        # self.axdict['nContacts'].legend()
        # self.axdict['nSplitBodyNodes'].legend()
        
        #Create custom legend for remote and local contacts
        options = self.generateOptions("Total")
        localContactsArtist = plt.Line2D((0,1),(0,0), linestyle='' ,**options['optMarker2'])
        remoteContactsArtist = plt.Line2D((0,1),(0,0),linestyle='' ,**options['optMarker3'])
        #Create legend from custom artist/label lists
        self.axdict['nContactsAllRL'].legend([localContactsArtist,remoteContactsArtist],
                                             ['Local', 'Remote'],**self.plotOpts["legendProps"])
        
        for ax in self.axdict.values():
            ax.grid(color=[0.4]*3,lw=dLineSettings["thin"])
            ax.margins(0.05,0.05)
            pS.defaultFormatAxes(ax)

        for f in self.figarr:
            f.tight_layout(pad=0.2)
            
       
    
    def plotSimDataAverage(self,SimDataAvg, optionsDict):
        
        defaultLineStyle = "-"
        self.axdict['MaxOverlap'].plot(SimDataAvg['SimulationTime'],
                                       SimDataAvg['MaxOverlap'], linestyle=defaultLineStyle,
                                       **optionsDict["optLine"]);
        
        optL = optionsDict["optLine"].copy()
        optL["color"]= pS.distinctColors["blackredgreen"][1]
        optL["label"] = "collision solver"
        self.axdict['allTimings'].plot(SimDataAvg['SimulationTime'],
                                       SimDataAvg['CollisionTime'], linestyle=defaultLineStyle,
                                       **optL);
        optL["label"] = "inclusion solver"
        optL["color"]= pS.distinctColors["blackredgreen"][0]
        self.axdict['allTimings'].plot(SimDataAvg['SimulationTime'],
                                       SimDataAvg['InclusionTime'], linestyle=defaultLineStyle,
                                       **optL);
        
        optL["color"]= pS.distinctColors["blackredgreen"][2]
        optL["label"] = "body communication"
        self.axdict['allTimings'].plot(SimDataAvg['SimulationTime'],
                                       SimDataAvg['BodyCommunicationTime'], linestyle=defaultLineStyle,
                                       **optL);
        
        optL = optionsDict["optLine"].copy()
        optL["label"] = "total time step"
        self.axdict['allTimings'].plot(SimDataAvg['SimulationTime'],
                                       SimDataAvg['TimeStepTime'], linestyle=defaultLineStyle,
                                       **optL);
        
        leg=self.axdict['allTimings'].legend(**self.plotOpts["legendProps"])
        for legobj in leg.legendHandles:
            legobj.set_linewidth(dLineSettings["thick"])
            
        self.axdict['TimeStepTime'].plot(SimDataAvg['SimulationTime'],
                                         SimDataAvg['TimeStepTime'],linestyle=defaultLineStyle, 
                                         **optionsDict["optLine"]);

        self.axdict['collisionTime'].plot(SimDataAvg['SimulationTime'],
                                         SimDataAvg['CollisionTime'],linestyle=defaultLineStyle, 
                                         **optionsDict["optLine"]); 
        
        
        self.axdict['inclusionTime'].plot(SimDataAvg['SimulationTime'],
                                         SimDataAvg['InclusionTime'],linestyle=defaultLineStyle, 
                                         **optionsDict["optLine"]); 
        self.axdict['bodyCommTime'].plot(SimDataAvg['SimulationTime'],
                                         SimDataAvg['BodyCommunicationTime'],linestyle=defaultLineStyle, 
                                         **optionsDict["optLine"]); 
        # average legend
        for t in ["TimeStepTime",'collisionTime','inclusionTime','bodyCommTime']:
            leg=self.axdict[t].legend(**self.plotOpts["legendProps"])
            for legobj in leg.legendHandles:
                legobj.set_linewidth(dLineSettings["thick"])
        
    def plotSimDataProcess(self,SimData, procIdx, optionsDict):
        
            from matplotlib.collections import LineCollection
            def getVals(allD):
                return allD[procIdx,:].T
            
            
            #Iterations
            self.axdict['nGlobalIterations'].plot(SimData['SimulationTime'],
                                                 getVals(SimData['nGlobalIterations']),
                                                 linestyle='',**optionsDict['optLine'] );
            
            #Convergence
            self.axdict['Converged'].plot(SimData['SimulationTime'],
                                          getVals(SimData['Converged']),
                                          linestyle='',**optionsDict['optMarker'] );
            
            #Contacts Number All
            self.axdict['nContactsAll'].plot(SimData['SimulationTime'],
                                             getVals(SimData['nContacts']),
                                             linestyle='',**optionsDict['optMarker'] );
            
            #Contacts Number remote Local
            self.axdict['nContactsAllRL'].plot(SimData['SimulationTime'],
                                               getVals(SimData['nContactsLocal']),
                                               linestyle='',
                                               **optionsDict['optMarker2'] );
            self.axdict['nContactsAllRL'].plot(SimData['SimulationTime'],
                                               getVals(SimData['nContactsRemote']),
                                               linestyle='',
                                               **optionsDict['optMarker3'] );
            
            #SplitNodes Number
            self.axdict['nSplitBodyNodes'].plot(SimData['SimulationTime'],
                                                getVals(SimData['nSplitBodyNodes']),
                                                linestyle='',
                                                **optionsDict['optMarker'] );
            
            #TimeStep Time
            self.axdict['TimeStepTime'].plot(SimData['SimulationTime'],
                                             getVals(SimData['TimeStepTime']),
                                             linestyle='-',
                                             **optionsDict['optLine']);
            
            #TimeStep Time
            optL=optionsDict['optLine'].copy()
            self.axdict['allTimings'].plot(SimData['SimulationTime'],
                                           getVals(SimData['TimeStepTime']),
                                           linestyle='-',
                                           **optL);
            optL["color"]= pS.distinctColors["blackredgreenlight"][1]
            self.axdict['allTimings'].plot(SimData['SimulationTime'],
                                           getVals(SimData['CollisionTime']),
                                           linestyle='-',**optL);
            optL["color"]= pS.distinctColors["blackredgreenlight"][0]
            self.axdict['allTimings'].plot(SimData['SimulationTime'],
                                           getVals(SimData['InclusionTime']),
                                           linestyle='-',**optL);
            optL["color"]= pS.distinctColors["blackredgreenlight"][2]
            self.axdict['allTimings'].plot(SimData['SimulationTime'],
                                           getVals(SimData['BodyCommunicationTime']),
                                           linestyle='-',**optL);
            
            #TimeStep Time
            optL=optionsDict['optLine'].copy()
            self.axdict['bodyCommTime'].plot(SimData['SimulationTime'],
                                             getVals(SimData['BodyCommunicationTime']),
                                             linestyle='-',**optL);
            self.axdict['collisionTime'].plot(SimData['SimulationTime'],
                                              getVals(SimData['CollisionTime']), 
                                              linestyle='-',**optL);
            self.axdict['inclusionTime'].plot(SimData['SimulationTime'],
                                              getVals(SimData['InclusionTime']), 
                                              linestyle='-', **optL);
            
            
            #MaxOverlap
            optL["alpha"] = 0.2
            self.axdict['MaxOverlap'].plot(SimData['SimulationTime'],
                                           getVals(SimData['MaxOverlap']),
                                           linestyle='-',**optL);
            
    def plotSimDataTotal(self, SimDataT, optionsDict):
        
        self.axdict['nContacts'].plot(SimDataT['SimulationTime'],
                                      SimDataT['nContacts'],linestyle='-',
                                      **optionsDict['optLine'] );
        
        
        optL = copy.copy(optionsDict['optLine'])
        optL['color']=pS.distinctColors["colors3light"][0]
        self.axdict['TotalStateEnergy'].plot(SimDataT['SimulationTime'],
                                             SimDataT['TotalStateEnergy'],
                                             linestyle='-',
                                             **optionsDict['optLine']);
        
        optL['color']=pS.distinctColors["colors3light"][1]
        optL['label']='Pot. Energy'
        self.axdict['TotalStateEnergy'].plot(SimDataT['SimulationTime'],
                                             SimDataT['TotalPotEnergy'], linestyle='-',
                                             **optL);
        optL['color']=pS.distinctColors["colors3light"][2]
        optL['label']='Kin. Energy'
        self.axdict['TotalStateEnergy'].plot(SimDataT['SimulationTime'],
                                             SimDataT['TotalKinEnergy'], 
                                             linestyle='-',**optL);

        self.axdict['TotalStateEnergy'].set_xlim((0,SimDataT['SimulationTime'][-1]))
        
        
        percBC = SimDataT['BodyCommunicationTime'] / SimDataT['TimeStepTime']
        percCL = SimDataT['CollisionTime'] / SimDataT['TimeStepTime']
        percIC = SimDataT['InclusionTime'] / SimDataT['TimeStepTime']
        polys = self.axdict['maxTimings'].stackplot(SimDataT['SimulationTime'] ,  
                                                    percBC, 
                                                    percCL, 
                                                    percIC,
                                                    colors = pS.distinctColors["colors3"],
                                                    edgecolor='none',
                                                );
        self.axdict['maxTimings'].set_ylim([0,1])
        
        # add legends
        legendProxies = []
        for poly in polys:
            legendProxies.append(plt.Rectangle((0, 0), 1, 1, 
                                               fc=poly.get_facecolor()[0],lw=dLineSettings["thin"]))
            
        leg=self.axdict['maxTimings'].legend(legendProxies, 
                                         ['body communication',
                                          'collision solver',
                                          'inclusion solver' ]
                                        ,**self.plotOpts["legendProps"])

        
        self.axdict['MaxOverlap'].plot(SimDataT['SimulationTime'],SimDataT['MaxOverlap'],**optionsDict['optLine']);
        
    def generateColorMap(self,l):
        jet = colorMap = plt.get_cmap('jet') 
        cNorm  = colors.Normalize(vmin=0, vmax=l)
        scalarMap = cm.ScalarMappable(norm=cNorm, cmap=colorMap)
        return scalarMap;
    
    def generateOptions(self,typ,scalarMap=None,proc=0):
        
        defaultLineSize = dLineSettings["thin"]
        defaultAlpha=1.0
        if(typ == 'Total'):
            label = 'total'
            colorVal = 'b'
        elif( typ == 'Average'):
            label='average'
            colorVal = 'b'
        elif( typ == 'Min'):
            label='min'
            colorVal = 'b'
        elif( typ == 'All'):
            label=None
            colorVal = 'k'
            defaultAlpha=0.05
            defaultLineSize = dLineSettings["extra-thin"]
        else:
            raise NameError("wrong typ")        
        
        optLine = {
            'linewidth':defaultLineSize, 
            'color':colorVal, 
            'label' : label,
            'zorder':1.5,
            'alpha':defaultAlpha
                  }
        
        optMarker = {    
            'marker' : '.',
            'markersize':0.4*dMarkerSize,#(processNr+1)*2+6, 
            'markeredgecolor' : 'none', 
            'markerfacecolor' : colorVal, 
            'markeredgewidth': dLineSettings["extra-thin"],
            'label' : label,
            'alpha':defaultAlpha,
            'zorder':1.5
                    }
        optMarker2 = {  
            'marker' : 'o' ,
            'markersize':0.4*dMarkerSize,#(processNr+1)*3+10,
            'markeredgecolor' : 'none', 
            'markerfacecolor': colorVal, 
            'markeredgewidth': dLineSettings["extra-thin"],
            'alpha':defaultAlpha,
            'zorder':1.5
                     }
        optMarker3 = {   
            'marker' : 's', 
            'markersize':0.4*dMarkerSize,#(processNr+1)*3+10,
            'markeredgecolor' : 'none', 
            'markerfacecolor': colorVal,  
            'markeredgewidth': dLineSettings["extra-thin"],
            'zorder':1.5,
            'alpha':defaultAlpha,
                     }
        optStep =  { 
            'marker' : '.', 
            'markersize':0.4*dMarkerSize, 
            'markeredgecolor' : 'none', 
            'markerfacecolor': colorVal, 
            'markeredgewidth': dLineSettings["extra-thin"],
            'where' : 'post',
            'label' : label,
            'zorder':1.5
                    }
        
        return {
            'optLine': optLine, 
            'optMarker': optMarker, 
            'optStep': optStep, 
            'optMarker2': optMarker2,
            'optMarker3': optMarker3
                }

        
    def combineSimData(self,simDataFiles):
        
        
        # add all states from processes
        simDataList = dict()
        for processNr, simDataFile in simDataFiles.items():
            cNT = self.generateColumnNames(simDataFile)
            dt = [(s,np.float64) for s in cNT]
               
            # load structured unit        
            simDataList[processNr] = np.loadtxt(simDataFile,dtype=dt, delimiter="\t", comments='#');
            
            if processNr == 0:
                origcNT = cNT
                print("SimulationTime: max:" , np.max(simDataList[0]['SimulationTime']))
                
            if(cNT != origcNT):
                raise NameError(str(cNT) + " is not the same as " + str(origcNT))
            
            
            # add pot energy (if not already existing)
            if not "TotalPotEnergy" in simDataList[processNr].dtype.names:
                
                simDataList[processNr] = recf.append_fields(
                                  simDataList[processNr], 
                                  "TotalPotEnergy" , 
                                  simDataList[processNr] ["TotalStateEnergy"] -
                                    simDataList[processNr] ["TotalKinEnergy"] 
                                  )
            
            
            
        #determine common number of rows for SimDataTotal (not all processes might have written equal amount of rows... -> shutdown)
        minRows = np.shape(simDataList[0])[0];
        for simData in simDataList.values():
            minRows = min(minRows, np.shape(simData)[0]);
        print("common rows: " + str(minRows));
        
        #build data tensor for all time series
        procIndices = sorted(list(simDataList.keys()))
        dataAll = {"SimulationTime" : simDataList[0]["SimulationTime"][0:minRows]}
        for t in origcNT :
            if t != "SimulationTime":
                data = np.ndarray((len(simDataList),minRows),dtype=np.float64);
                for i,p in enumerate(procIndices):
                    data[i,:] = simDataList[p][t][0:minRows]
                dataAll[t] = data

        
        #make dictionary for total maximal values
        SimDataTotal = {"SimulationTime" : dataAll["SimulationTime"]}
        for key in ['TotalStateEnergy','TotalPotEnergy',
                    'TotalKinEnergy','nContacts']:
            SimDataTotal[key] = np.sum(dataAll[key],axis=0)
        
        for key in ['MaxOverlap']:
            SimDataTotal[key] = np.amax(dataAll[key],axis=0)
            
        # find indices of processes with maximal timestep time
        indices=np.argmax(dataAll["TimeStepTime"],axis=0)
        
        for key in ['TimeStepTime','CollisionTime',
                    'InclusionTime',
                    'BodyCommunicationTime']:
            SimDataTotal[key] = dataAll[key][indices,range(minRows)] 
            # select the values for the process with max timtestep time
        
        
        
        # make dictionary for average values over all processes
        SimDataAvg = {"SimulationTime" : dataAll["SimulationTime"]}
        for key in ['TimeStepTime','CollisionTime','InclusionTime',
                    'BodyCommunicationTime','AvgIterTime','MaxOverlap',
                    'TotalTimeProx','IterTimeProx']:
            SimDataAvg[key] = np.average(dataAll[key],axis=0);
        
        # make minimum values
        SimDataMin = {"SimulationTime" : dataAll["SimulationTime"]}
        for key in ['TimeStepTime','CollisionTime','InclusionTime',
                        'BodyCommunicationTime','AvgIterTime','MaxOverlap',
                        'TotalTimeProx','IterTimeProx']:
            SimDataMin[key]   = np.amin(dataAll[key],axis=0 )
        
        
        simDataList["All"]     = dataAll
        simDataList['Average'] = SimDataAvg;
        simDataList['Total']   = SimDataTotal;
        simDataList['Min']     = SimDataMin;
        
        # print some statistics
        summedTimeStepTime = np.sum(SimDataAvg['TimeStepTime'])/60
        summedMaxTimeStepTime = np.sum(SimDataTotal['TimeStepTime'])/60
        globalTotalTime = (simDataList[0]["GlobalTime"][-1] - simDataList[0]["GlobalTime"][0])/60
        print("Summed TimeStepTime  [min]: " + str(summedTimeStepTime))
        print("Summed max. TimeStepTime  [min]: " + str(summedMaxTimeStepTime))
        print("Global Time (proc 0) [min]: " + str(globalTotalTime))
  
        return procIndices, simDataList;
    
    
#     def plotComparisionGUI(self,simDataFile):
#         #compare to other SimData
#         columnNames = self.generateColumnNames(simDataFile)
#         dt = [(s,np.float64) for s in columnNames]
#         # load structured unit        
#         SimDataCompare = np.loadtxt(simDataFile,dtype=dt, delimiter="\t", comments='#')
#         self.plotSimData(SimDataCompare)
        #SimDataCompare['nContacts'] == simDataList['Total']['nContacts']
        
#     def plotSimData(self,SimData):
        
#         optionsDict = { 'optLine' : {  'linewidth':1, 
#                                        'color':'k',
#                                        'label' : 'GUI'
#                                     } ,
            
#                         'optMarker': {   'markersize':7, 
#                                          'markeredgecolor':'none',
#                                          'markerfacecolor':'k',
#                                          'label' : 'GUI'
#                                      },
#                         'optStep' :  {   'markersize':7, 
#                                          'markeredgecolor':'none',
#                                          'markerfacecolor':'k',
#                                          'label' : 'GUI',
#                                          'where' : 'post'
#                              }
#                       }
#         #Energy
#         self.axdict['TotalStateEnergy'].plot(SimData['SimulationTime'],SimData['TotalStateEnergy'],'-',**optionsDict['optLine']);
        
#         #Iterations
#         self.axdict['nGlobalIterations'].plot(SimData['SimulationTime'],SimData['nGlobalIterations'],'-', **optionsDict['optLine']); 
#         #Convergence
#         self.axdict['Converged'].plot(SimData['SimulationTime'],SimData['Converged'],'.', **optionsDict['optMarker']);
        
#         #Contacts Number
#         self.axdict['nContacts'].plot(SimData['SimulationTime'],SimData['nContacts'],'.', **optionsDict['optMarker']);
    
#         #TimeStep Time
#         #axdict['TimeStepTime'].plot(SimData['SimulationTime'],SimData['TimeStepTime'],'-', **optionsLinePlot);
        
#         #MaxOverlap
#         axdict['MaxOverlap'].plot(SimData['SimulationTime'],SimDataTotal['MaxOverlap'],'k-', lw=3);
          
#end class

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np

# x = np.linspace(0,10,31)

# fig = plt.figure()
# ax = fig.add_subplot(1,1,1)

# #Create custom artists
# simArtist = plt.Line2D((0,1),(0,0), color='k', marker='o', linestyle='')
# anyArtist = plt.Line2D((0,1),(0,0), color='k')

# #Create legend from custom artist/label lists
# ax.legend([simArtist,anyArtist],
#           ['Simulation', 'Analytic'])

# plt.show()

In [ ]:

bodyRange = [0,500]
timeRange = [0,5]

#For single file which contains multiple bodies
#dynState = ReadInSimFile("./SimFiles/SimDataRECORDGUI_0/SimulationState.sim",[0,10],[0,10],False)
#For multiple file which contains 1 body!
#dynState = ReadInSimFileMulti("./SimFiles/SimDataRECORD_0/", [0,10] , [0,10], "SimDataBody-1-(\d*)\.sim" ,True)
#print(dynState)

folderNumber= 0
filePath = os.path.join("../GlobalFolder/SimFiles/" , "SimDataRECORDMPI_%d" % folderNumber, "./SimState.sim");
#filePath = "/media/zfmgpu/Data/GabrielNuetzi/SimFiles/SimFilesBrutus/Avalanche1M-DriftC/first/SimStateRenderTest.sim"

dynState = SimFileReader.ReadInSimFile( filePath ,timeRange,bodyRange,False)

plotData(dynState)
res = plotData3D(dynState);


In [ ]:
localFolderPath = "../LocalFolder/"
#localFolderPath = "/media/zfmgpu/Data/GabrielNuetzi/SimFiles/SimFilesBrutus/Avalanche1M-DriftC/first/"

simDataRelPath = "./SimDataRECORDMPI_0/SimData.dat"
regExProcessFolder=".*ProcessMPI_(\d*).*"
simDataPathGUI=""
p = PlotSimDataMPI();
p.plot(localFolderPath,simDataRelPath,regExProcessFolder,simDataPathGUI)

Plot MPI Topology
---

In [6]:
# makeSymlinks
import glob2;

# Globs all files with the pattern  fileGlobPattern
# and builds for each found file a new symlink where the
# file name is determined by match/substituting the filePath 
# with the regex: filePathRegex and substitution pattern: subPattern
# reEnumerateStart defines if an additiona counter should be added and where to start!
# and saves a new symlink in the outputFolder for each file found.
def  makeSymlinks(fileGlobPattern, filePathRegex, subPattern, outputFolder, reEnumerateStart=-1):
    
    files = glob2.glob(fileGlobPattern);
    newFileNames = [  re.sub(filePathRegex,subPattern,f) for f in files]
    files = sorted(zip(newFileNames,files))
    
    os.makedirs(outputFolder,exist_ok=True);
    
    # make symlinks to all TopologyInfos in outputFolder
    counter = reEnumerateStart;
    for s,f in files:
        basename = os.path.basename(f);
        head, ext = os.path.splitext(basename);
        if reEnumerateStart <= 0:
            simLinkFile = s + ext;
        else:
            simLinkFile = s + "-" + str(counter) + ext;
            counter+=1;
        simLinkPath = os.path.join(outputFolder,simLinkFile) 
        print("Link: " + simLinkFile + " --- to ---> " + simLinkPath)
        try:
            os.symlink(f, simLinkPath );
        except FileExistsError:
            continue;
    
# makeSymlinkSequence("/media/zfmgpu/Data1/GabrielNuetzi/SimFiles/SimFilesBrutus/Avalanche1M-RunDownTrees"+
#                     "/*/ProcessMPI_0/SimDataRECORDMPI_0/TopologyInfo_*.xml", 
#                     r".*Trees/(\d)*/.*_(\d)*.*", 
#                     r"TopologyInfo_\1-\2",
#                     "/media/zfmgpu/Data1/GabrielNuetzi/SimFiles/SimFilesBrutus/Avalanche1M-RunDownTrees/AllTopologies/")

In [7]:
# plot Topology MPI
import re, os
import numpy as np
import math

from vispy import app
from vispy import scene
from vispy.visuals.collections import PointCollection
from vispy.visuals.transforms import STTransform, NullTransform, BaseTransform, MatrixTransform

from transforms3d import quaternions, axangles

import Tools.Parsers.TopologyInfo as tpI

def generateGridBoxLines(minP = np.array([-1.0,-1.0,-1.0]), 
              maxP = np.array([1.0,1.0,1.0]), 
              R_KI = None,
              subdivs=None, boundaryBox = True):
    
    centerPos = (maxP + minP)/2.0;
    extent = (maxP - minP);
    
    if boundaryBox:
        nLines = 12;
    else:
        nLines = 0;
    # for each subdiv a quad is (4 lines) is drawn
    if subdivs is not None:
       nLines += (subdivs[0]+1)*(subdivs[1]+1) + (subdivs[1]+1)*(subdivs[2]+1) + (subdivs[2]+1)*(subdivs[0]+1) - 4*3;

    lines = np.zeros((nLines*2,3), dtype=np.float32);
    
    # plot lines
    if boundaryBox:
        lines[0:24,0:3] = 0.5* np.array([ (-1, -1, -1),
                                        (1, -1, -1),
                                        #
                                        (1, -1, -1),
                                        (1, 1, -1),
                                        #
                                        (1, 1, -1),
                                        (-1, 1, -1), 
                                        #
                                        (-1, 1, -1),
                                        (-1, -1, -1), # end of first quad
                                        #
                                        (-1, -1, 1),
                                        (1, -1, 1),
                                        #
                                        (1, -1, 1),
                                        (1, 1, 1),
                                        #
                                        (1, 1, 1),
                                        (-1, 1, 1), 
                                        #
                                        (-1, 1, 1),
                                        (-1, -1, 1), # end of second quad
                                        #
                                        (-1, -1, -1),
                                        (-1, -1, 1),
                                        #
                                        (1, -1, -1),
                                        (1, -1, 1),
                                        #
                                        (-1, 1, -1),
                                        (-1, 1, 1), 
                                        #
                                        (1, 1, -1),
                                        (1, 1, 1)]); # end of vertical lines
    
    
    # add subdivs if necessary

    if subdivs is not None:
        counter = 12*2; # start after the cube lines
        dxyz = np.array([1.0 / subdivs[0],1.0 / subdivs[1],1.0 / subdivs[2]]);
        
        # y
        # |    +   +   +   
        # | *  *   *   *   *
        # | *  *   *   *   *
        # |    +   +   +  
        # ----------------> x
        #  * come first , then the +
        
        # make x lines inside (*)
        for k in range(1,int(subdivs[1])):
            for l in range(0,int(subdivs[2])+1):
                lines[counter:counter+2,0:3] = np.array([ (-0.5, -0.5 + k*dxyz[1],  -0.5 + l*dxyz[2]),
                                                          (0.5, -0.5 + k*dxyz[1],  -0.5 + l*dxyz[2])  ]);
                counter+=2;
        # make the x lines (+)
        for k in (0,subdivs[1]):
            for l in range(1,int(subdivs[2])):
                lines[counter:counter+2,0:3] = np.array([ (-0.5, -0.5 + k*dxyz[1],  -0.5 + l*dxyz[2]),
                                                          (0.5, -0.5 + k*dxyz[1],  -0.5 + l*dxyz[2])  ]);
                counter+=2;
        
        
        # make y lines inside (*)     
        for k in range(1,int(subdivs[2])):
            for l in range(0,int(subdivs[0])+1):
                lines[counter:counter+2,0:3] = np.array([ (-0.5+ l*dxyz[0],  -0.5,  -0.5 + k*dxyz[2]),
                                                          (-0.5+ l*dxyz[0],  0.5,  -0.5 + k*dxyz[2])  ]);
                counter+=2;
        # make the y lines (+)
        for k in (0,subdivs[2]):
            for l in range(1,int(subdivs[0])):
                lines[counter:counter+2,0:3] = np.array([ (-0.5+ l*dxyz[0],  -0.5,  -0.5 + k*dxyz[2]),
                                                          (-0.5+ l*dxyz[0],  0.5,  -0.5 + k*dxyz[2])  ]);
                counter+=2;
                
        # make z lines inside
        for k in range(1,int(subdivs[0])):
            for l in range(0,int(subdivs[1])+1):
                lines[counter:counter+2,0:3] = np.array([ (-0.5+k*dxyz[0],  -0.5+l*dxyz[1],  -0.5),
                                                          (-0.5+k*dxyz[0],  -0.5+l*dxyz[1],  0.5)    ]); 
                counter+=2;
        # make the z lines (+)
        for k in (0,subdivs[0]):
            for l in range(1,int(subdivs[1])):
                lines[counter:counter+2,0:3] = np.array([ (-0.5+k*dxyz[0],  -0.5+l*dxyz[1],  -0.5),
                                                          (-0.5+k*dxyz[0],  -0.5+l*dxyz[1],  0.5)    ]); 
                counter+=2;

    lines[...] *= extent;
    if R_KI is not None:
        lines = np.dot(lines,R_KI.T); # [p1.T; p2.T] * R.transpose
        lines[...] += R_KI.dot(centerPos);    
    else:
        lines[...] += centerPos;
        
    return lines;
            

        

        

class PlotTopologyMPI:
    
    def __init__(self):
        self.canvasStore = []
        pass
            
    def makeCanvas(self,i,oneWindow=False):
        if oneWindow and len(self.canvasStore):
            return self.canvasStore[-1];
        else:
            
            canvas = scene.SceneCanvas(size=(800, 600), show=True, keys='interactive', bgcolor="white")
            canvas.title = "TopoInfo " + i;
            print("Make canvas: %s" % canvas.title)
            vb = canvas.central_widget.add_view(); # create view box

            vb.camera = scene.TurntableCamera(elevation=30, azimuth=30, up='+z')

#             fig = plt.figure();
#             fig.title = "TopoInfo " + i;
#             ax = plt.gca();
            ax=None
            fig=None
        
        self.canvasStore.append((canvas,vb,fig,ax))
        return self.canvasStore[-1];           
    
    def makeFigure(self,title):
        fig = plt.figure();
        fig.title = title;
        ax = plt.gca();
        return (fig,ax)
    
    def plot(self, folderPath, filePathRegex,  index = None,
             loadArgs = {"loadPoints" : False, "loadHistogram" : False},
             oneWindow = False, 
             plotArgs = dict()):
        
        self.folderPath=folderPath;
        self.filePathRegex=filePathRegex
        self.fileIndex = index;
        
        self.loadData(self.folderPath, self.filePathRegex, self.fileIndex, **loadArgs);
         
        # init colormap
        self.colorMap = self.generateColorMap(max(self.topoInfos.keys()), min(self.topoInfos.keys()) )
        
        # Plot stuff

        if self.fileIndex is None:
            self.fileIndex = self.topoInfos.keys();
            
        # Plot TopoInfo for index = 0 
        l = []
        for i,t in self.topoInfos.items():
            
            can,vb,fig,ax = self.makeCanvas(os.path.basename(t.filePath) if not oneWindow else "all", oneWindow);
            self.plotTopoInfo(vb,ax,i,self.topoInfos[i],**plotArgs);
            l.append((can,vb,self.topoInfos[i]))

        return l
        
    def run(self):
        app.run();
        
    def loadData(self,folderPath, filePathRegex, fileIndex=None, **kargs):
        
        if fileIndex is None:
            loadAll=True;
        else:
            loadAll=False;
            
        # Looop through process folder
        self.topoInfos = dict();
        files = [ os.path.join(folderPath,f) for f in os.listdir(folderPath) if os.path.isfile( os.path.join(folderPath,f) ) ]
        
        for f in files:
            res = re.match(filePathRegex,f)
            if(res):
                
                fileNr = int(res.group(1));
                # shortcut
                extO = lambda o : self.extractOptPerIdx(o,fileNr)
                
                if not loadAll:
                    load = fileNr in fileIndex;
                else:
                    load = True;
                if load:
                    print("Load file: ", f)

                    # extract all opts
                    args = dict([(k,extO(v)) for k,v in kargs.items()])
                        
                    t = TopologyXMLParser.parseTopologyInfoXML(f,False,**args);
                    if fileNr in self.topoInfos:
                        print("Key %i for file %s already in map, not added!" % (fileNr,f))
                    self.topoInfos[fileNr] = t;
        
        print("TopoInfos: " + str(self.topoInfos))
        
    # extract options per index (all options can be dictionaries)
    def extractOptPerIdx(self,opts,i):
        if isinstance(opts,dict) :
            if "default" in opts: 
                # if dictionary contains default thens 
                #it is considered a dict with opts per index
                return opts[i] if i in opts else opts['default']
            
        return opts
        
    def plotTopoInfo(self,viewBox, axes, i,topoInfo,
                     plotPoints=False,plotSubDivs=False,plotHistogram=False,
                     plotCellIdx=False,
                     plotAxis = True,
                     R_KI = None,
                     topoName="Topo %i",
                     textOffset=0.2,
                     plotLabel = True,
                     pointStride=1,
                     plotOpts=None,
                     shrinkEps=0):
        
        # shortcut
        extO = lambda o : self.extractOptPerIdx(o,i)
        
        rootNode = scene.Node(parent=viewBox.scene,name=topoName%i)
        if R_KI is not None:
            rootNode.transform = scene.MatrixTransform(extO(R_KI))

        topoColor = self.colorMap.to_rgba(i)[0:3]
        
        opts={ 'topo':{'lc':topoColor,'mc':topoColor, "lw" : 3,"method":"gl"} , 
               'points':{'edge_color':None, 'face_color':topoColor, 'edge_width':0.5 , 'symbol':'o', 'size':1.0}, 
               'subdivs' :{'ls':'+'}
            }
        
        # update dictionaries
        import collections
        def update(d, u):
            for k, v in u.items():
                if isinstance(v, collections.Mapping):
                    r = update(d.get(k, {}), v)
                    d[k] = r
                else:
                    d[k] = u[k]
            return d
            
        if plotOpts is not None:
            update(opts,extO(plotOpts))
        
        # Plot axis
        if extO(plotAxis):
            axis = scene.visuals.XYZAxis(parent=rootNode,antialias = True);
        
        if( isinstance(topoInfo.topo , tpI.Grid) ):
            # Plotting GRID
            g = topoInfo.topo;
            
            if(extO(plotSubDivs)):
                subdivs = g.processDim;
            else:
                subdivs = None;
                
            # plot topo
            l= generateGridBoxLines(g.aabb.min, g.aabb.max , R_KI = g.A_IK, subdivs=subdivs);
            scene.visuals.Line(pos=l,color=opts['topo']['lc'],
                               parent=rootNode,
                               name="grid",
                               antialias = True,
                               connect='segments', method=opts["topo"]["method"], width=opts["topo"]["lw"])
 
        elif( isinstance(topoInfo.topo , tpI.KdTree) ):
            
            # Plotting KDTREE
            kdTree = topoInfo.topo;
            levels = kdTree.leafs.keys();
            
            cm = self.generateColorMap(min=min(levels),max=max(levels));
            
            # plot root box
            lines = generateGridBoxLines(minP=kdTree.aabb.min, maxP=kdTree.aabb.max, R_KI = kdTree.A_IK);
            colors = np.ones((len(lines),4), dtype=np.float32); colors[...,0:3]*= opts['topo']['mc'];
                
            if extO(plotSubDivs):
                # add all leafs
                for level,leafs in kdTree.leafs.items():
                    for leaf in leafs:
                        l = generateGridBoxLines(minP=leaf.aabb.min+shrinkEps, maxP=leaf.aabb.max-shrinkEps, R_KI = kdTree.A_IK);
                        color = opts['topo']['lc'];
                        c = np.ones((len(l),4), dtype=np.float32); c[...,0:3]= color;#cm.to_rgba(level);
                        colors = np.concatenate(( colors,  c));
                        lines = np.concatenate(( lines,  l));
                        
                        if(plotCellIdx):
                            scene.visuals.Text(str(leaf.idx), font_size=14, bold=True, parent=rootNode, color=color, 
                                               pos = kdTree.A_IK.dot(leaf.aabb.center())[0:3]);

#                 # add all lines for the aabbs
#                 counter = 0;
#                 for level, aabbs in kdTree.aabbTree.items():
#                     if level == 0:
#                         continue;
#                     for aabb in aabbs:
#                         l = generateGridBoxLines(minP=aabb['min'], maxP=aabb['max']);
#                         c = np.ones((len(l),4), dtype=np.float32); c*= cm.to_rgba(level);
#                         colors = np.concatenate(( colors,  c));
#                         lines = np.concatenate(( lines,  l));
#                         counter+=1;
               
            scene.visuals.Line(pos=lines,color=colors,
                               parent=rootNode,
                               name="kdTree",
                               connect='segments', 
                               antialias = True,
                               method=opts["topo"]["method"], width=opts["topo"]["lw"])    
        
            
        else:
            raise NameError("TopoInfo has unsupported type: " + str(type(topoInfo)) + " for plotting!")
        
        # plot points ==============
        
        if extO(plotPoints) and topoInfo.points is not None and np.size(topoInfo.points) :
            topoInfo.points = topoInfo.points[0:-1:pointStride]
            if(isinstance(topoInfo.topo , tpI.KdTree)):
                topoInfo.points =  topoInfo.points.dot(topoInfo.topo.A_IK.T) # transform points back to I frame
            mk = scene.visuals.Markers(parent=rootNode)
            mk.set_data( topoInfo.points, **opts["points"])
        # ============================
            
        # plot histogram =============
        if extO(plotHistogram) and topoInfo.nearestDists is not None:
            fig,axes = self.makeFigure(topoName)
            axes.hist( topoInfo.nearestDists.distances , bins=200, normed=True);
            axes.set_title(r"Nearest Distances ($\mu = %f$, $\sigma = %f$)" % (topoInfo.nearestDists.mean, topoInfo.nearestDists.stdDev))
        # ============================

        #plot label
        if extO(plotLabel):
            r = topoInfo.topo.aabb.min - topoInfo.topo.aabb.max
            off = r/np.linalg.norm(r)*textOffset
            scene.visuals.Text(topoName % i, parent=rootNode, color=opts['topo']['lc'], 
                               pos = topoInfo.topo.A_IK.dot(topoInfo.topo.aabb.min+off))
        # ============================

        
    def generateColorMap(self,max, min=0):
        jet = colorMap = plt.get_cmap('spectral') 
        cNorm  = colors.Normalize(vmin=min, vmax=max)
        scalarMap = cm.ScalarMappable(norm=cNorm, cmap=colorMap,)
        return scalarMap;

SImulation Plots
===
Local Simulation
---

In [ ]:
# 
folderPath = "../LocalFolderKd/ProcessMPI_0/SimDataRECORDMPI_0"
filePathRegex = ".*TopologyInfo_(\d*).xml"
p = PlotTopologyMPI();

index=[5];
p.plot(folderPath,
       filePathRegex,index, 
       loadArgs = {"loadPoints" : True, "loadHistogram" : False}
       ,oneWindow=True,
       plotArgs={"plotPoints":True,"plotSubDivs":True, "plotCellIdx":True, "topoName": "Topo1.%i", "textOffset":0.1});
p.run()


Good And Bad Split Plots
===

In [ ]:
# Good and Bad Splits
from vispy.visuals.transforms import (STTransform, PerspectiveTransform,
                                   NullTransform, AffineTransform,
                                   TransformCache)
from vispy.util.transforms import perspective, translate, rotate

folderPath = "KdTreeSplits/"
filePathRegex = ".*/(?:BadSplit|GoodSplit)_(\d*)\.xml"
p = PlotTopologyMPI();

index=[1,2];
plots = p.plot(folderPath,
       filePathRegex,index, 
       loadArgs = {"loadPoints" : True, "loadHistogram" : True}
       ,oneWindow=False,
       plotArgs={"plotPoints":True,"plotSubDivs":True, "plotCellIdx":False, "plotHistogram" : True, 
                 "plotAxis":False, "topoName": "Topo1.%i",
                 "textOffset":0.1, "pointStride":2, "shrinkEps":0.0001 , 
                 "plotOpts" : {"topo": {"lw":3},
                               "points" : {"edge_color" : None, "face_color": [0,0,0,0.9] , "size":3} } });

for pl in plots:
    topo = pl[2]
    vb = pl[1]
    can = pl[0]

    # vb.camera.transform =  AffineTransform()
    vb.camera.elevation = 0
    vb.camera.azimuth =0
    vb.camera.center=[0.64,0,0]
    vb.camera.scale_factor = 0.5
    can.size = (1700,1200)
    print(vb.describe_tree(True))

    class grabber:
        
        def __init__(self, can, name):
            self.can = can;
            self.name = name
        def grab(self,event):
            from vispy.gloo.util import _screenshot
            from vispy.io import imsave
            im = _screenshot((0, 0, self.can.size[0], self.can.size[1]))
            imsave(self.name, im)

    can.events.key_press.connect(grabber(can, 
                                         os.path.join(folderPath,'screenShot%s.tif' % os.path.basename(topo.filePath))
                                        ).grab)

p.run()


Avalanche1M-DriftC-2
---

In [ ]:
#
plt.close('all')
folderPath = "/media/zfmgpu/Data/GabrielNuetzi/SimFiles/SimFilesBrutus/Avalanche1M-DriftC-2/second/ProcessMPI_0/SimDataRECORDMPI_0"
filePathRegex = ".*TopologyInfo_(\d*).xml"
index=range(1,2);
p = PlotTopologyMPI(folderPath,filePathRegex,index);


# for i, t in enumerate(p.topoInfos.values()):
#      fname=os.path.join(folderPath,"PointCloud_%i.txt" % i);
#      np.savetxt(fname, t.points, fmt='%f', delimiter=' ', newline='\n', header='', footer='', comments='# ');


p.plot(index,ax=None,oneWindow=True,plotPoints=True);

Avalanche1M-FillUp1
---

In [ ]:
#
plt.close('all')
folderPath = "/media/zfmgpu/Data/GabrielNuetzi/SimFiles/SimFilesBrutus/Avalanche1M-FillUp/1/ProcessMPI_0/SimDataRECORDMPI_0"
filePathRegex = ".*TopologyInfo_(\d*).xml"
index=range(1,5);
p = PlotTopologyMPI(folderPath,filePathRegex,index);
p.plot(index,ax=None,oneWindow=True,plotPoints=True,plotSubDivs=False);

In [ ]:
#
localFolderPath = "/media/zfmgpu/Data/GabrielNuetzi/SimFiles/SimFilesBrutus/Avalanche1M-FillUp/1/"

simDataRelPath = "./SimDataRECORDMPI_0/SimData.dat"
regExProcessFolder=".*ProcessMPI_(\d*).*"
simDataPathGUI=""
p = PlotSimDataMPI(localFolderPath,simDataRelPath,regExProcessFolder,simDataPathGUI);

p.plot(oneWindow=False)

Avalanche1M-FillUp3
---

In [ ]:
#
localFolderPath = "/media/zfmgpu/Data/GabrielNuetzi/SimFiles/SimFilesBrutus/Avalanche1M-FillUp3/1/"

simDataRelPath = "./SimDataRECORDMPI_0/SimData.dat"
regExProcessFolder=".*ProcessMPI_(\d*).*"
simDataPathGUI=""
p = PlotSimDataMPI(localFolderPath,simDataRelPath,regExProcessFolder,simDataPathGUI);

p.plot(oneWindow=False)

Avalanche1M-FillUp4
---

In [ ]:
#
localFolderPath = "/media/zfmgpu/Data/GabrielNuetzi/SimFiles/SimFilesBrutus/Avalanche1M-FillUp4/1/"

simDataRelPath = "./SimDataRECORDMPI_0/SimData.dat"
regExProcessFolder=".*ProcessMPI_(\d*).*"
simDataPathGUI=""
p = PlotSimDataMPI(localFolderPath,simDataRelPath,regExProcessFolder,simDataPathGUI);

p.plot(oneWindow=False)

In [ ]:
#
plt.close('all')
for key,value in p.simDataList['Average'].items():
    print(key)
    fd = np.array(value);
    p.simDataList['Average'][key] = np.average(fd,axis=0);
    
plt.plot(p.simDataList['Average']['MaxOverlap'])

In [ ]:
#
plt.close('all')
localFolderPath = "/media/zfmgpu/Data/GabrielNuetzi/SimFiles/SimFilesBrutus/Avalanche1M-FillUp4/2/"

simDataRelPath = "./SimDataRECORDMPI_0/SimData.dat"
regExProcessFolder=".*ProcessMPI_(\d*).*"
simDataPathGUI=""
p = PlotSimDataMPI(localFolderPath,simDataRelPath,regExProcessFolder,simDataPathGUI);

p.plot(oneWindow=False)

In [ ]:
#
plt.close('all')
localFolderPath = "/media/zfmgpu/Data/GabrielNuetzi/SimFiles/SimFilesBrutus/Avalanche1M-FillUp5/1/"

simDataRelPath = "./SimDataRECORDMPI_0/SimData.dat"
regExProcessFolder=".*ProcessMPI_(\d*).*"
simDataPathGUI=""
p = PlotSimDataMPI(localFolderPath,simDataRelPath,regExProcessFolder,simDataPathGUI);

p.plot(oneWindow=False)

In [ ]:
#
bodyRange = [0,50]
timeRange = [0,5]

#For single file which contains multiple bodies
#dynState = ReadInSimFile("./SimFiles/SimDataRECORDGUI_0/SimulationState.sim",[0,10],[0,10],False)
#For multiple file which contains 1 body!
#dynState = ReadInSimFileMulti("./SimFiles/SimDataRECORD_0/", [0,10] , [0,10], "SimDataBody-1-(\d*)\.sim" ,True)
#print(dynState)

folderNumber= 0
localFolderPath = "/media/zfmgpu/Data/GabrielNuetzi/SimFiles/SimFilesBrutus/Avalanche1M-FillUp5/1/"
simFileRelPath = "./SimState.sim"
filePath = os.path.join(localFolderPath , simFileRelPath);
#filePath = "/media/zfmgpu/Data/GabrielNuetzi/SimFiles/SimFilesBrutus/Avalanche1M-DriftC/first/SimStateRenderTest.sim"

dynState = SimFileReader.ReadInSimFile( filePath ,timeRange,bodyRange,False)

plotData(dynState)
res = plotData3D(dynState);


In [ ]:
#
plt.close('all')

folderPath = "/media/zfmgpu/Data/GabrielNuetzi/SimFiles/SimFilesBrutus/Avalanche1M-FillUp/1/ProcessMPI_0/SimDataRECORDMPI_0"
filePathRegex = ".*TopologyInfo_(\d*).xml"
index=range(1,5);
p = PlotTopologyMPI(folderPath,filePathRegex,index);
p.plot(index,ax=None,oneWindow=True,plotPoints=True,plotSubDivs=False);

AvalancheFillUp5
---

In [ ]:
#
plt.close('all')
localFolderPath = "/media/zfmgpu/Data/GabrielNuetzi/SimFiles/SimFilesBrutus/Avalanche1M-FillUp5/1/"

simDataRelPath = "./SimDataRECORDMPI_0/SimData.dat"
regExProcessFolder=".*ProcessMPI_(\d*).*"
simDataPathGUI=""
p = PlotSimDataMPI(localFolderPath,simDataRelPath,regExProcessFolder,simDataPathGUI);

p.plot(oneWindow=False)

AvalancheRunDownTrees
---

SimData Plots

In [ ]:
#
plt.close('all')
localFolderPath = "/media/zfmgpu/Data1/GabrielNuetzi/SimFiles/SimFilesBrutus/Avalanche1M-RunDownTrees/1/"
simDataRelPath = "./SimDataRECORDMPI_0/SimData.dat"
regExProcessFolder=".*ProcessMPI_(\d*).*"
simDataPathGUI=""
p = PlotSimDataMPI(localFolderPath,simDataRelPath,regExProcessFolder,simDataPathGUI);
p.plot(oneWindow=False)

Topology Plots

In [ ]:
# make symlinks
localFolderPath = "/media/zfmgpu/Data1/GabrielNuetzi/SimFiles/SimFilesBrutus/Avalanche1M-RunDownTrees";

In [ ]:
makeSymlinks(localFolderPath+ "/*/ProcessMPI_0/SimDataRECORDMPI_0/TopologyInfo_*.xml", 
                    r".*Trees/(\d)*/.*_(\d)*.*", 
                    r"TopologyInfo_\1\2",
                    "/media/zfmgpu/Data1/GabrielNuetzi/SimFiles/SimFilesBrutus/Avalanche1M-RunDownTrees/AllTopologies/",
                     reEnumerateStart=1)

In [ ]:
folderPath =  localFolderPath +  "/AllTopologies/"
filePathRegex = ".*TopologyInfo_\d*-(\d*).xml"

index=range(3,4);
p = PlotTopologyMPI(folderPath,filePathRegex,index);
p.plot(index,oneWindow=True,plotPoints=True,plotSubDivs=True);

AvalancheRunDownEmpty
===

In [ ]:
folderPath = "/media/zfmgpu/MyDataRaid/GabrielNuetzi/SimFilesEulerBrutus/Avalanche1M-RunDownEmpty/Avalanche1M-RunDownEmpty.%i";

In [ ]:
simDataRelPath = "./SimDataRECORDMPI_0/SimData.dat"
regExProcessFolder=".*ProcessMPI_(\d*).*"
simDataPathGUI=""

p = PlotSimDataMPI();
p.plot(folderPath % 0,simDataRelPath,regExProcessFolder,simDataPathGUI, oneWindow=False)
p.plot(folderPath % 1,simDataRelPath,regExProcessFolder,simDataPathGUI, oneWindow=False)
p.save("./figure_%i.pdf")

In [ ]:
p.save("./figure_%i.jpg")

In [ ]:
filePathRegex = ".*TopologyInfo_(\d*).xml"
p = PlotTopologyMPI();

# index=range(1,13);
# p.plot(folderPath % 0 + "/ProcessMPI_0/SimDataRECORDMPI_0",
#        filePathRegex,index, 
#        loadArgs = {"loadPoints" : False, "loadHistogram" : False}
#        ,oneWindow=True,
#        plotArgs={"plotPoints":False,"plotSubDivs":True, "topoName":"Topo1.%i", "textOffset":0.1});

index=range(1,22);
p.plot(folderPath % 0 + "/ProcessMPI_0/SimDataRECORDMPI_0",
       filePathRegex,index, 
       loadArgs = {"loadPoints" : False, "loadHistogram" : False}
       ,oneWindow=True,
       plotArgs={"plotPoints":False,"plotSubDivs":True, "topoName":"Topo1.%i", "textOffset":0.1});
p.run()

AvalancheRunDownTrees-KdTree
===

In [ ]:
folderPath = "/media/zfmgpu/MyDataRaid/GabrielNuetzi/SimFilesEulerBrutus/Avalanche1M-RunDownTrees-KdTree/Avalanche1M-RunDownTrees-KdTree.%i";

In [ ]:
simDataRelPath = "./SimDataRECORDMPI_0/SimData.dat"
regExProcessFolder=".*ProcessMPI_(\d*).*"
simDataPathGUI=""

p = PlotSimDataMPI();
p.plot(folderPath % 0,simDataRelPath,regExProcessFolder,simDataPathGUI, oneWindow=False)
p.save("./figureKdTreeSim_%i.pdf")

In [ ]:
filePathRegex = ".*TopologyInfo_(\d*).xml"
p = PlotTopologyMPI();
index=range(1,14);
p.plot(folderPath % 0 + "/ProcessMPI_0/SimDataRECORDMPI_0",
       filePathRegex,index, 
       loadArgs = {"loadPoints" : False, "loadHistogram" : False}
       ,oneWindow=True,
       plotArgs={"plotPoints":True,"plotSubDivs":True, "plotCellIdx":False, "topoName":"Topo1.%i", "textOffset":0.1});
p.run()

Avalanche-Tree-Study-P-0
===

In [28]:
studyFolderPath = "/media/zfmgpu/MyDataRaid/GabrielNuetzi/SimFilesEulerBrutus/" \
"SimulationStudies/SimFiles/Avalanche1M-Trees-Study"

In [29]:
folderPath = os.path.join(studyFolderPath, "Avalanche1M-Trees-Study-P-0/Avalanche1M-Trees-Study-P-0.0")

In [ ]:
# for plot in thesis
from vispy.util.transforms import *

filePathRegex = ".*TopologyInfo_(\d*).xml"
p = PlotTopologyMPI();
index=[1,3,5,7,9,11,13,15,17,19,21];
p.plot(folderPath + "/ProcessMPI_0/SimDataRECORDMPI_0",
       filePathRegex,index, 
       loadArgs = {"loadPoints" : {1:True,21:True,"default":False}, "loadHistogram" : False},
       oneWindow=True,
       plotArgs={"plotPoints":{1:True,21:True,"default":False},"plotSubDivs":{1:False,21:True,"default":False}, 
                 "plotCellIdx":False, "topoName":"Topo-%i", 
                 "plotHistogram" : False,"plotLabel":False,
                 "plotOpts":{"topo":{"lw":2,"method":"gl"}},
                 "pointStride" : 300,
                 "R_KI" : rotate(axis=[0,1,0],angle=-30),
                 "textOffset":0.3});
p.run()


In [30]:
simDataRelPath = "./SimDataRECORDMPI_0/SimData.dat"
regExProcessFolder=".*ProcessMPI_(\d*).*"
simDataPathGUI=""

p = PlotSimDataMPI();
p.load(folderPath,
        simDataRelPath,
        regExProcessFolder)

SimDataFiles from 383 processes: 
SimulationTime: max: 1.0078
common rows: 504
Summed TimeStepTime  [min]: 71.8873345855
Summed max. TimeStepTime  [min]: 72.0499066667
Global Time (proc 0) [min]: 716.009265


In [31]:
plt.close("all")
p.plot(plotOpts = dict(showTitle=False))

/opt/python3.4Env/lib/python3.4/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


In [ ]:
p.save("./figureTree-P-0-%i.pdf")

In [33]:
p.save("./figureTree-P-0-%i.jpg", dpi=400)

TypeError: save() got an unexpected keyword argument 'dpi'

Consolidate all data of all simulation of this simulation study
===

In [ ]:

import glob2,os

fileName = "CombinedSimData.dat.gz"
skipIfExists = True;


studyFolderPath = "/media/zfmgpu/MyDataRaid/GabrielNuetzi/SimFilesEulerBrutus/" \
"SimulationStudies/SimFiles/Avalanche1M-Trees-Study"

studyPaths = glob2.glob(studyFolderPath+"/*-Study-*")
simPaths=[]
for i,p in enumerate(studyPaths):
    simPaths.append( glob2.glob(p+"/*-Study-*") )


for i,sims in enumerate(simPaths):
    
    studyFolder = studyPaths[i]
    print("Consoldating for study folder: %s" % studyFolder)
    
    combinedFiles = []
    for simPath in sims:
        path = os.path.join(simPath,fileName)
        print("Consoldating for simulation : %s ---> %s" % (simPath , path))
        combinedFiles.append(path)
        
        if os.path.exists(path) and skipIfExists:
            continue
    
        p = PlotSimDataMPI();
        folderPath = simPath
        simDataRelPath = "./SimDataRECORDMPI_0/SimData.dat"
        regExProcessFolder=".*ProcessMPI_(\d*).*"
        simDataPathGUI=""
        
        try:
            p.load(  folderPath,
                     simDataRelPath,
                     regExProcessFolder,
                     simDataPathGUI)

            p.saveCombined(path)
        except Exception as e:
            print("Exception occured: ", e)
            combinedFiles.pop() # remove file
            continue
        
    # combine all files of the simulations of this study into one
    path = os.path.join(studyFolder ,fileName)
    print("Consoldating for all sims : %s ---> %s" % (studyFolder , path))
    if os.path.exists(path) and skipIfExists:
        continue
        
    if len(combinedFiles) == 1:
        # only make a link!
        relPath = os.path.relpath(combinedFiles[0],studyFolder)
        os.symlink(relPath,path)
    elif combinedFiles:
        p = PlotSimDataMPI();
        p.loadCombined(combinedFiles) # load a list of combined files and join them
        p.saveCombined(path)
    else:
        raise ValueError("no combined files!")